## Competative Social Media Analysis

**Explore Data Science Academy** wants to develop a marketing startegy to ensure that all social media content posted reaches the inteded target market or audiance and that our social media market are growing in engagement and also analysing our growing social media engagement as compared to the competitors.

#### Import Standard libraries

In [84]:
import re
import csv
import pandas as pd

import selenium
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as bs

# for visualization
import seaborn as sns
import matplotlib.pyplot as plt

# for extraction
from geograpy import extraction
import nltk
nltk.download('maxent_ne_chunker')

# for organization
from collections import Counter

import time
from datetime import datetime


[nltk_data] Downloading package maxent_ne_chunker to /home/nano-
[nltk_data]     ai/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


#### Configurations

In [85]:
import warnings
warnings.filterwarnings('ignore')

#Disable chrome Notifications
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")

#Install latest chrome drivers
#driver = webdriver.chrome(service=Service(ChromeDriverManager().install()))

#Thanks stackoverflow @pythonjar
#chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

#specify the path to chromedriver.exe (download and save on your computer)
browser = webdriver.Chrome('/home/nano-ai/chromedriver' , chrome_options=chrome_options)

### Automatic Login to Social Media

In [86]:
#Get the Company's  URL from User
# page = input("Enter the Company Linkedin URL: ")

#See if existing user credential file exists or create one 
try:
    f= open("linkedin_credentials.txt","r")
    contents = f.read()
    username = contents.replace("=",",").split(",")[1]
    password = contents.replace("=",",").split(",")[3]
except:
    f= open("linkedin_credentials.txt","w+")
    username = input('Enter your linkedin username: ')
    password = input('Enter your linkedin password: ')
    f.write("username={}, password={}".format(username,password))
    f.close()


#### Navigate to facebook page/group

In [87]:
#accessing Chromedriver
# browser = webdriver.Chrome('chromedriver')
browser.maximize_window()
time.sleep(2)

#Open login page
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Enter login info:
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)
elementID.submit()

In [92]:
#Open txt file
with open("linkedin_users.txt") as f:
    lines = f.readlines()
    for page in lines:
        #Go to company post webpage
        browser.get(page)
        
        #Simulate scrolling to capture all posts
        SCROLL_PAUSE_TIME = 1.5

        # Get scroll height
        last_height = browser.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
            #Check out page source code
            company_page = browser.page_source  

            #Use Beautiful Soup to get access tags
            linkedin_soup = bs(company_page.encode("utf-8"), "html")
            linkedin_soup.prettify()

            #Find the post blocks
            containers = linkedin_soup.findAll("div",{"class":"occludable-update ember-view"})
        
            # Lists that we will iterate to
            post_dates = []
            post_texts = []
            post_likes = []
            post_comments = []
            post_shares = []
            media_links = []
            media_type = []
            company = []
            followers = []

            #Looping through the posts and appending them to the lists
            for container in containers:

                #Try function to make sure its a post and not a promotion
                try:
                    posted_date = container.find("span",{"class":"visually-hidden"})
                    text_box = container.find("div",{"class":"feed-shared-text relative feed-shared-update-v2__commentary"})
                    text = text_box.find("span",{"dir":"ltr"})
                    new_likes = container.findAll("span", {"class":"social-details-social-counts__reactions-count"})
                    new_comments = container.findAll("li", {"class": "social-details-social-counts__item social-details-social-counts__comments social-details-social-counts__item--with-social-proof"})
                    company_name = container.findAll("span" , {"class":"feed-shared-actor__name t-14 t-bold hoverable-link-text t-black"})
                    num_followers = container.findAll("span" , {"feed-shared-actor__description t-12 t-normal t-black--light"})


                    #Appending date and text to lists
                    post_dates.append(posted_date.text.strip())
                    post_texts.append(text_box.text.strip())


                    #Determining media type and collecting relevant info for each type
                    try:
                        video_box = container.findAll("div",{"class": "feed-shared-update-v2__content feed-shared-linkedin-video ember-view"})
                        video_link = video_box[0].find("video", {"class":"vjs-tech"})
                        media_links.append(video_link['src'])
                        media_type.append("Video")
                    except:
                        try:
                            image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                            image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image feed-shared-image__image--constrained lazy-image ember-view"})
                            media_links.append(image_link['src'])
                            media_type.append("Image")
                        except:
                            try:
                                image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                                image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image lazy-image ember-view"})
                                media_links.append(image_link['src'])
                                media_type.append("Image")
                            except:
                                try:
                                    article_box = container.findAll("div",{"class": "feed-shared-article__description-container"})
                                    article_link = article_box[0].find('a', href=True)
                                    media_links.append(article_link['href'])
                                    media_type.append("Article")
                                except:
                                    try:
                                        video_box = container.findAll("div",{"class": "feed-shared-external-video__meta"})          
                                        video_link = video_box[0].find('a', href=True)
                                        media_links.append(video_link['href'])
                                        media_type.append("Youtube Video")   
                                    except:
                                        try:
                                            poll_box = container.findAll("div",{"class": "feed-shared-update-v2__content overflow-hidden feed-shared-poll ember-view"})
                                            media_links.append("None")
                                            media_type.append("Other: Poll, Shared Post, etc")
                                        except:
                                            media_links.append("None")
                                            media_type.append("Unknown")



                    #Getting Video Views. (The folling three lines prevents class name overlap)
                    view_container2 = set(container.findAll("li", {'class':["social-details-social-counts__item"]}))
                    view_container1 = set(container.findAll("li", {'class':["social-details-social-counts__reactions","social-details-social-counts__comments social-details-social-counts__item"]}))
                    result = view_container2 - view_container1

                    view_container = []
                    for i in result:
                        view_container += i

                    try:
                        post_shares.append(view_container[1].text.strip())

                    except:
                        post_shares.append(0)


                    #Appending likes and comments if they exist
                    try:
                        post_likes.append(new_likes[0].text.strip())
                    except:
                        post_likes.append(0)
                        pass

                    try:
                        post_comments.append(new_comments[0].text.strip())
                    except:                                                           
                        post_comments.append(0)

                        pass
                    try:
                        company.append(company_name[0].text.strip())

                    except:
                        company.append(0)

                    try:
                        followers.append(num_followers[0].text.strip())

                    except:
                        followers.append(0)

                except:
                    pass
                
            #Stripping non-numeric values
            comment_count = []
            for i in post_comments:
                s = str(i).replace('Comment','').replace('s','').replace(' ','')
                comment_count += [s]
            
            data = {
            "Company": company,
            "Date Posted": post_dates,
            "Media Type": media_type,
            "Post Text": post_texts,
            "Post Likes": post_likes,
            "Post Comments": comment_count,
            "Post Shares": post_shares,
            "Media Links": media_links,
            "Followers":followers
            }
            df = pd.DataFrame(data)
            #Exporting as csv file to program folder
            df.to_csv("{}_posts.csv".format('linkedin_data'), encoding='utf-8', index=False)
            df

In [94]:
df

,Company,Date Posted,Media Type,Post Text,Post Likes,Post Comments,Post Shares,Media Links,Followers
0,WeThinkCode,4 days ago,Youtube Video,"This morning #WeThinkCode CEO, Nyari Samushong...",0,4comment,8 shares,https://www.youtube.com/watch?v=umhqUl0UHR8,"10,696 followers"
1,WeThinkCode,2 weeks ago,Image,We are excited to announce that several of our...,0,5comment,5 comments,https://media-exp1.licdn.com/dms/image/C5622AQ...,"10,696 followers"
2,WeThinkCode,1 week ago,Article,"""We risk leaving talented youth behind in our ...",0,0,1 share,https://gadget.co.za/tech-advances-must-leave-...,"10,696 followers"
3,WeThinkCode,3 weeks ago,Image,"Last week our CEO, Nyari Samushonga, spoke at ...",0,0,5 shares,https://media-exp1.licdn.com/dms/image/C5622AQ...,"10,696 followers"
4,WeThinkCode,1 month ago,"Other: Poll, Shared Post, etc","""Every day, African businesses harness ingenui...",31,0,2 shares,None,"10,696 followers"
5,WeThinkCode,2 weeks ago,Article,"Mood, because the 2020 cohort is done and star...",0,2comment,2 comments,https://media.giphy.com/media/6uOKby3tWy4yXwTa...,"10,696 followers"
6,WeThinkCode,1 month ago,Image,"Across all the #WeThinkCode campuses, the firs...",98,5comment,5 comments,https://media-exp1.licdn.com/dms/image/C5622AQ...,"10,696 followers"
7,WeThinkCode,2 months ago,Article,Are you a developer with ideas for new and inn...,9,0,0,https://stitch.money/hackathon/,"10,696 followers"
8,WeThinkCode,3 months ago,Image,"#WeThinkCode alum, Vus'muzi Sibanyoni recently...",0,0,0,https://media-exp1.licdn.com/dms/image/C4D22AQ...,"10,696 followers"
9,WeThinkCode,1 month ago,Image,"Our CEO, Nyari Samushonga, will be speaking at...",0,0,4 shares,https://media-exp1.licdn.com/dms/image/C5622AQ...,"10,696 followers"


### Check page source code

In [ ]:
# #Check out page source code
# company_page = browser.page_source  


# #Use Beautiful Soup to get access tags
# linkedin_soup = bs(company_page.encode("utf-8"), "html")
# linkedin_soup.prettify()

# #Find the post blocks
# containers = linkedin_soup.findAll("div",{"class":"occludable-update ember-view"})

### Data Scraping

In [ ]:
# # Lists that we will iterate to
# post_dates = []
# post_texts = []
# post_likes = []
# post_comments = []
# post_shares = []
# media_links = []
# media_type = []
# company = []
# followers = []

# #Looping through the posts and appending them to the lists
# for container in containers:
    
#     #Try function to make sure its a post and not a promotion
#     try:
#         posted_date = container.find("span",{"class":"visually-hidden"})
#         text_box = container.find("div",{"class":"feed-shared-text relative feed-shared-update-v2__commentary"})
#         text = text_box.find("span",{"dir":"ltr"})
#         new_likes = container.findAll("span", {"class":"social-details-social-counts__reactions-count"})
#         new_comments = container.findAll("li", {"class": "social-details-social-counts__item social-details-social-counts__comments social-details-social-counts__item--with-social-proof"})
#         company_name = container.findAll("span" , {"class":"feed-shared-actor__name t-14 t-bold hoverable-link-text t-black"})
#         num_followers = container.findAll("span" , {"feed-shared-actor__description t-12 t-normal t-black--light"})

        
#         #Appending date and text to lists
#         post_dates.append(posted_date.text.strip())
#         post_texts.append(text_box.text.strip())


#         #Determining media type and collecting relevant info for each type
#         try:
#             video_box = container.findAll("div",{"class": "feed-shared-update-v2__content feed-shared-linkedin-video ember-view"})
#             video_link = video_box[0].find("video", {"class":"vjs-tech"})
#             media_links.append(video_link['src'])
#             media_type.append("Video")
#         except:
#             try:
#                 image_box = container.findAll("div",{"class": "feed-shared-image__container"})
#                 image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image feed-shared-image__image--constrained lazy-image ember-view"})
#                 media_links.append(image_link['src'])
#                 media_type.append("Image")
#             except:
#                 try:
#                     image_box = container.findAll("div",{"class": "feed-shared-image__container"})
#                     image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image lazy-image ember-view"})
#                     media_links.append(image_link['src'])
#                     media_type.append("Image")
#                 except:
#                     try:
#                         article_box = container.findAll("div",{"class": "feed-shared-article__description-container"})
#                         article_link = article_box[0].find('a', href=True)
#                         media_links.append(article_link['href'])
#                         media_type.append("Article")
#                     except:
#                         try:
#                             video_box = container.findAll("div",{"class": "feed-shared-external-video__meta"})          
#                             video_link = video_box[0].find('a', href=True)
#                             media_links.append(video_link['href'])
#                             media_type.append("Youtube Video")   
#                         except:
#                             try:
#                                 poll_box = container.findAll("div",{"class": "feed-shared-update-v2__content overflow-hidden feed-shared-poll ember-view"})
#                                 media_links.append("None")
#                                 media_type.append("Other: Poll, Shared Post, etc")
#                             except:
#                                 media_links.append("None")
#                                 media_type.append("Unknown")



#         #Getting Video Views. (The folling three lines prevents class name overlap)
#         view_container2 = set(container.findAll("li", {'class':["social-details-social-counts__item"]}))
#         view_container1 = set(container.findAll("li", {'class':["social-details-social-counts__reactions","social-details-social-counts__comments social-details-social-counts__item"]}))
#         result = view_container2 - view_container1

#         view_container = []
#         for i in result:
#             view_container += i

#         try:
#             post_shares.append(view_container[1].text.strip())

#         except:
#             post_shares.append(0)

        
#         #Appending likes and comments if they exist
#         try:
#             post_likes.append(new_likes[0].text.strip())
#         except:
#             post_likes.append(0)
#             pass

#         try:
#             post_comments.append(new_comments[0].text.strip())
#         except:                                                           
#             post_comments.append(0)
            
#             pass
#         try:
#             company.append(company_name[0].text.strip())
            
#         except:
#             company.append(0)
            
#         try:
#             followers.append(num_followers[0].text.strip())

#         except:
#             followers.append(0)
    
#     except:
#         pass


In [ ]:
# #Stripping non-numeric values
# comment_count = []
# for i in post_comments:
#     s = str(i).replace('Comment','').replace('s','').replace(' ','')
#     comment_count += [s]

In [80]:
# data = {
#     "Company": company,
#     "Date Posted": post_dates,
#     "Media Type": media_type,
#     "Post Text": post_texts,
#     "Post Likes": post_likes,
#     "Post Comments": comment_count,
#     "Post Shares": post_shares,
#     "Media Links": media_links,
#     "Followers":followers
# }

# df = pd.DataFrame(data)

In [82]:
post_likes

[]

In [81]:
df.tail(50)

,Company,Date Posted,Media Type,Post Text,Post Likes,Post Comments,Post Shares,Media Links,Followers


In [ ]:
data['Company'][0]

### Data Preprocessing

#### Convert Dataframe to CSV File

In [69]:
#Exporting as csv file to program folder
df.to_csv("{}_posts.csv".format(df['Company'][0]), encoding='utf-8', index=False)

#### Save csv file to Coud Storage (Aws RDS)